In [50]:
%pip install eep153_tools
%pip install python_gnupg
%pip install -U gspread_pandas
#load in file from class
def format_id(id,zeropadding=0):
    """Nice string format for any id, string or numeric.

    Optional zeropadding parameter takes an integer
    formats as {id:0z} where
    """
    if pd.isnull(id) or id in ['','.']: return None

    try:  # If numeric, return as string int
        return ('%d' % id).zfill(zeropadding)
    except TypeError:  # Not numeric
        return id.split('.')[0].strip().zfill(zeropadding)
    except ValueError:
        return None

data_url = "https://docs.google.com/spreadsheets/d/12Z4n8HbFZRYvH6B-D8EDLDibRiL50zNMlSBLMJ41C1o/"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [53]:
import pandas as pd
from eep153_tools.sheets import read_sheets

#create recipes df
recipes = read_sheets(data_url, sheet="recipes")
recipes = (recipes
           .assign(parent_foodcode = lambda df: df["parent_foodcode"].apply(format_id),
                   ingred_code = lambda df: df["ingred_code"].apply(format_id))
           .rename(columns={"parent_desc": "recipe"}))


#recipes[recipes["recipe"].str.contains("Vegetable lasagna", case=False)]
recipes.head()
#recipes.shape


,parent_foodcode,recipe,ingred_code,ingred_desc,ingred_wt
0,11340000,"Imitation milk, non-soy, sweetened",43543,"Milk, imitation, non-soy",100.0
1,11460150,"Yogurt, frozen, NS as to flavor, lowfat milk",1298,"Yogurt, frozen, flavors other than chocolate, ...",100.0
2,11460160,"Yogurt, frozen, chocolate, lowfat milk",1117,"Yogurt, plain, low fat, 12 grams protein per 8...",81.8
3,11460160,"Yogurt, frozen, chocolate, lowfat milk",19166,"Cocoa, dry powder, unsweetened, processed with...",5.2
4,11460160,"Yogurt, frozen, chocolate, lowfat milk",19335,"Sugars, granulated",13.0


In [54]:
#check the data types
recipes.dtypes

parent_foodcode     object
recipe              object
ingred_code         object
ingred_desc         object
ingred_wt          float64
dtype: object

In [55]:
#convert ingred_desc and recipe columns to str before filtering for Vegan foods 
#recipes["ingred_desc"] = recipes["ingred_desc"].astype(str)
#recipes["recipe"] = recipes["recipe"].astype(str)

#recipes.head()

In [56]:
#List of non-vegan keywords
NON_VEGAN_KEYWORDS = [
    "beef", "pork", "chicken", "turkey", "fish", "seafood", "shellfish", "shrimp", "crab","crabs",
    "lamb", "goat", "duck", "goose", "tuna", "salmon", "cod", "bacon", "ham",
    "shellfish", "lobster", "mussels", "oysters", "scallops", "octopus", "eel",
    "organ meat", "milk", "cheese", "butter", "cream","ice cream", "yogurt", "whey",
    "casein", "lactose", "ghee", "buttermilk", "egg", "eggs", "mayo", "mayonnaise", "albumen",
    "albumin", "lysozyme", "ovomucoid", "ovomucin", "ovovitellin", "honey",
    "bee pollen", "royal jelly", "propolis", "shellac", "confectioner’s glaze",
    "carmine", "cochineal", "lard", "tallow", "suet", "gelatin", "collagen",
    "isinglass", "bone broth", "bone stock", "fish sauce", "oyster sauce",
    "shrimp paste", "worcestershire sauce", "anchovies", "rennet", "pepsin",
    "bone char", "vitamin d3", "lanolin", "omega-3 fish oil", "caseinate",
    "lecithin (egg)", "cysteine", "l-cysteine", "glycerin (animal)",
    "glycerol (animal)", "stearic acid (animal)", "tallowate", "sodium tallowate",
    "capric acid", "caprylic acid"
]

def contains_non_vegan(text: str) -> bool:
    """Returns True if `text` contains any non-vegan keywords, otherwise False."""
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in NON_VEGAN_KEYWORDS)

#goes throught all rows in INGRED_DESC  AND RECIPE columns and filters out all rows that contain vegan foods
# In my first attempt I only filtered over the ingred_desc column. However, I was seeing non-vegan items in the recipe column so I decided to filter over the recipe column too.

def is_vegan(row) -> bool:

    #make sure the columns are strings
    recipe_text = str(row["recipe"])       
    ingred_text = str(row["ingred_desc"]) 
    
    # If either column contains non-vegan keywords, return False
    if contains_non_vegan(recipe_text) or contains_non_vegan(ingred_text):
        return False
    return True

def filter_vegan_ingredients(df):
    """
    Returns a new DataFrame containing only rows
    that are vegan according to the 'recipe' and 'ingred_desc' columns.
    """
    mask = df.apply(is_vegan, axis=1)
    return df[mask]

In [58]:
vegan_df = filter_vegan_ingredients(recipes)
vegan_df.head()

,parent_foodcode,recipe,ingred_code,ingred_desc,ingred_wt
32,11830100,"Hot chocolate / Cocoa, dry mix, not reconstituted",14192,"Beverages, Cocoa mix, powder",100.0
33,11830115,"Hot chocolate / Cocoa, dry mix, no sugar added...",14196,"Beverages, Cocoa mix, no sugar added, powder",100.0
179,13170000,Baked Alaska,2047,"Salt, table",0.6
180,13170000,Baked Alaska,4610,"Margarine, regular, 80% fat, composite, stick,...",42.2
181,13170000,Baked Alaska,14411,"Water, tap, drinking",0.3


In [59]:
#check to see if the filter_vegan_ingredients worked

def check_non_vegan_ingredients(df):
    mask = df.apply(lambda row: contains_non_vegan(str(row["recipe"])) or 
                                contains_non_vegan(str(row["ingred_desc"])), 
                    axis=1)
    return df[mask]

non_vegan_in_vegan_df = check_non_vegan_ingredients(vegan_df)
if non_vegan_in_vegan_df.empty:
    print("✅ No non-vegan ingredients found.")
else:
    print("⚠️ Found non-vegan ingredients in the filtered DataFrame:")
    display(non_vegan_in_vegan_df)


✅ No non-vegan ingredients found.


In [60]:
#my idea is to filter over only foods that athelecs would eat. Organic, low fat etc.